<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.leaky_relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.078125
Current benign train loss: 2.396028518676758

Current batch: 100
Current benign train accuracy: 0.234375
Current benign train loss: 2.060962438583374

Current batch: 200
Current benign train accuracy: 0.3203125
Current benign train loss: 1.8758670091629028

Current batch: 300
Current benign train accuracy: 0.296875
Current benign train loss: 1.8571115732192993

Current batch: 400
Current benign train accuracy: 0.4296875
Current benign train loss: 1.7283927202224731

Current batch: 500
Current benign train accuracy: 0.265625
Current benign train loss: 1.842607855796814

Current batch: 600
Current benign train accuracy: 0.421875
Current benign train loss: 1.6218130588531494

Current batch: 700
Current benign train accuracy: 0.3828125
Current benign train loss: 1.6520658731460571

Current batch: 800
Current benign train accuracy: 0.3671875
Current benign train loss: 1.6489062309265137

Current batch: 900
Current


Current batch: 800
Current benign train accuracy: 0.8125
Current benign train loss: 0.42684224247932434

Current batch: 900
Current benign train accuracy: 0.90625
Current benign train loss: 0.23776556551456451

Current batch: 1000
Current benign train accuracy: 0.859375
Current benign train loss: 0.37035638093948364

Current batch: 1100
Current benign train accuracy: 0.875
Current benign train loss: 0.33251088857650757

Total benign train accuarcy: 85.29866666666666
Total benign train loss: 493.4960304647684

[ Test epoch: 5 ]

Test accuarcy: 83.49
Test average loss: 0.005008695381879807
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.3190750777721405

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.25317689776420593

Current batch: 200
Current benign train accuracy: 0.875
Current benign train loss: 0.4002731144428253

Current batch: 300
Current benign train accuracy: 0.8515


Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2643384039402008

Current batch: 300
Current benign train accuracy: 0.890625
Current benign train loss: 0.3303816020488739

Current batch: 400
Current benign train accuracy: 0.8671875
Current benign train loss: 0.39917901158332825

Current batch: 500
Current benign train accuracy: 0.875
Current benign train loss: 0.40421172976493835

Current batch: 600
Current benign train accuracy: 0.8828125
Current benign train loss: 0.40317296981811523

Current batch: 700
Current benign train accuracy: 0.9140625
Current benign train loss: 0.24022962152957916

Current batch: 800
Current benign train accuracy: 0.8515625
Current benign train loss: 0.3304232656955719

Current batch: 900
Current benign train accuracy: 0.890625
Current benign train loss: 0.3039645254611969

Current batch: 1000
Current benign train accuracy: 0.9140625
Current benign train loss: 0.266481876373291

Current batch: 1100
Current benign tra


Current batch: 1000
Current benign train accuracy: 0.90625
Current benign train loss: 0.2547916769981384

Current batch: 1100
Current benign train accuracy: 0.921875
Current benign train loss: 0.2645571231842041

Total benign train accuarcy: 91.062
Total benign train loss: 305.55933675915

[ Test epoch: 16 ]

Test accuarcy: 85.96
Test average loss: 0.004494989624619484
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.16703811287879944

Current batch: 100
Current benign train accuracy: 0.90625
Current benign train loss: 0.26755788922309875

Current batch: 200
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2123340517282486

Current batch: 300
Current benign train accuracy: 0.921875
Current benign train loss: 0.25346091389656067

Current batch: 400
Current benign train accuracy: 0.90625
Current benign train loss: 0.22069838643074036

Current batch: 500
Current benign train accuracy: 0.9453125
C


Current batch: 400
Current benign train accuracy: 0.8984375
Current benign train loss: 0.27350708842277527

Current batch: 500
Current benign train accuracy: 0.90625
Current benign train loss: 0.27865737676620483

Current batch: 600
Current benign train accuracy: 0.90625
Current benign train loss: 0.22851474583148956

Current batch: 700
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1619662493467331

Current batch: 800
Current benign train accuracy: 0.890625
Current benign train loss: 0.2914048433303833

Current batch: 900
Current benign train accuracy: 0.921875
Current benign train loss: 0.2516701817512512

Current batch: 1000
Current benign train accuracy: 0.90625
Current benign train loss: 0.2503809928894043

Current batch: 1100
Current benign train accuracy: 0.9453125
Current benign train loss: 0.23365257680416107

Total benign train accuarcy: 91.65333333333334
Total benign train loss: 286.95073318481445

[ Test epoch: 22 ]

Test accuarcy: 85.66
Test average

...


Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.01365224365144968

Current batch: 700
Current benign train accuracy: 0.984375
Current benign train loss: 0.039503660053014755

Current batch: 800
Current benign train accuracy: 0.9921875
Current benign train loss: 0.010153357870876789

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.008875226601958275

Current batch: 1000
Current benign train accuracy: 0.9921875
Current benign train loss: 0.042210616171360016

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.01687474735081196

Total benign train accuarcy: 99.582
Total benign train loss: 16.239583950897213

[ Test epoch: 72 ]

Test accuarcy: 92.54
Test average loss: 0.002878830238431692
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0014890274032950401

Current batch: 100
Current benign train accuracy: 1.0
Current ben


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0011038073571398854

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.009064632467925549

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.002138768322765827

Current batch: 400
Current benign train accuracy: 0.9921875
Current benign train loss: 0.012325979769229889

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.003947753459215164

Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.008958686143159866

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.005371811334043741

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.003078578971326351

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.005809028632938862

Current batch: 1000
Current benign train accuracy: 1.0
Curr


Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.0014417079510167241

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.0012982124462723732

Total benign train accuarcy: 99.89266666666667
Total benign train loss: 5.734309392399155

[ Test epoch: 83 ]

Test accuarcy: 93.14
Test average loss: 0.00262136302664876
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.003238713601604104

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.002650722861289978

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.008579587563872337

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0003850634675472975

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0005753806908614933

Current batch: 500
Current benign train accuracy: 1.0
Current 


Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.003198545891791582

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0010577966459095478

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0007160663953982294

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.0013134032487869263

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.004974310752004385

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.0018404086586087942

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.0016130059957504272

Total benign train accuarcy: 99.92733333333334
Total benign train loss: 4.727030570909847

[ Test epoch: 89 ]

Test accuarcy: 93.15
Test average loss: 0.0025882635824382304
Model Saved!

[ Train epoch: 90 ]

Current batch: 0
Current benign train accuracy: 1.0
Current ben


Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.017917968332767487

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.001101393485441804

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0022210106253623962

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0007261950522661209

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0020699824672192335

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0011830548755824566

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.004670536145567894

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.0010134641779586673

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.0009442593436688185

Current batch: 1000
Current benign train accuracy: 1.0
Curre